In [1]:
pip install tweepy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Import Libraries
from textblob import TextBlob
import sys
import tweepy
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import jsonpickle
import os
import nltk
import pycountry
import re
import string

from wordcloud import WordCloud, STOPWORDS
from PIL import Image
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from langdetect import detect
from nltk.stem import SnowballStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
# Authentication
api_key = "QFYvRqbGk7lIzPhz6ogOGPXIC"
api_secret = "tBawjRIk0wbElquMhncdvTVkDaRyvu0b7lu51cwBBppV0il1P5"
bearerToken = "AAAAAAAAAAAAAAAAAAAAADg6lAEAAAAAJ15yNl701isfu8Mzq5Wd7Wv4NuU%3D2FvkEtCPihsjlM4EhmqTWkYV4y6KUSXxTUdfL9wQtPJLxR9x62"

auth = tweepy.OAuth2BearerHandler(bearerToken)
api = tweepy.API(auth)

In [4]:
#Download tweets to json file
def percentage(part,whole):
     return 100 * float(part)/float(whole)

query = "agriculture"
environment = "development"
fromDate = "202201010000"
tweets = tweepy.Cursor(api.search_full_archive,label=environment,query=query,fromDate=fromDate).items()

In [5]:
# Testing 7 days search
tweets = tweepy.Cursor(api.search_tweets,q="agriculture -is:retweet", lang="en", result_type="mixed", count=100).items(1)

In [6]:
# Save tweets
pd.json_normalize([t._json for t in tweets]).to_csv("agriculture_tweets_2022.csv")

In [7]:
tweets_df = pd.read_csv("agriculture_tweets_2022.csv")

In [8]:
tweets_df.head()

,Unnamed: 0,created_at,id,id_str,text,truncated,source,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,...,user.profile_text_color,user.profile_use_background_image,user.has_extended_profile,user.default_profile,user.default_profile_image,user.following,user.follow_request_sent,user.notifications,user.translator_type,user.withheld_in_countries
0,0,Tue Jan 10 09:58:52 +0000 2023,1612750814493409280,1612750814493409280,Wonderful community participation to #SaveSoil...,True,"<a href=""http://twitter.com/download/android"" ...",NaN,NaN,NaN,...,333333,True,False,False,False,NaN,NaN,NaN,none,[]


In [9]:
tweets_df.describe()

,Unnamed: 0,id,id_str,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,in_reply_to_screen_name,geo,coordinates,...,user.favourites_count,user.utc_offset,user.time_zone,user.statuses_count,user.lang,user.profile_sidebar_border_color,user.profile_text_color,user.following,user.follow_request_sent,user.notifications
count,1.0,1.000000e+00,1.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0
mean,0.0,1.612751e+18,1.612751e+18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,12735.0,NaN,0.0,333333.0,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,0.0,1.612751e+18,1.612751e+18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,12735.0,NaN,0.0,333333.0,NaN,NaN,NaN
25%,0.0,1.612751e+18,1.612751e+18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,12735.0,NaN,0.0,333333.0,NaN,NaN,NaN
50%,0.0,1.612751e+18,1.612751e+18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,12735.0,NaN,0.0,333333.0,NaN,NaN,NaN
75%,0.0,1.612751e+18,1.612751e+18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,12735.0,NaN,0.0,333333.0,NaN,NaN,NaN
max,0.0,1.612751e+18,1.612751e+18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,12735.0,NaN,0.0,333333.0,NaN,NaN,NaN
